In [3]:
import sys

import numpy as np
import numba as nb
from numba import prange, njit, jit

import scipy.stats
from scipy.stats import pearsonr
import scipy.special as sc
from scipy.io import mmread

import statsmodels.api as sm

import numba_scipy.special

from sklearn.linear_model import LinearRegression

import pandas as pd

import matplotlib.pyplot as plt

# load data

## expression data

In [4]:
exp_data=mmread('data/extract/HumanLiver.data.counts.mm').toarray()
with open('data/extract/HumanLiver.data.col','r') as f: exp_data_col=[i.strip().strip('"') for i in f.read().split()]
with open('data/extract/HumanLiver.data.row','r') as f: exp_data_row=[i.strip().strip('"') for i in f.read().split()]
assert exp_data.shape==(len(exp_data_row),len(exp_data_col))

In [5]:
exp_data,exp_data.shape

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 (20007, 8444))

In [6]:
exp_data_row[:5],exp_data_col[:5]

(['RP11-34P13.7', 'FO538757.2', 'AP006222.2', 'RP4-669L17.10', 'RP5-857K21.4'],
 ['P1TLH_AAACCTGAGCAGCCTC_1',
  'P1TLH_AAACCTGTCCTCATTA_1',
  'P1TLH_AAACCTGTCTAAGCCA_1',
  'P1TLH_AAACGGGAGTAGGCCA_1',
  'P1TLH_AAACGGGGTTCGGGCT_1'])

## cluster info 

In [7]:
exp_data_meta=pd.read_csv('data/extract/HumanLiver.metadata.tsv',sep='\t')
exp_data_meta.head()

,total_counts,total_features,orig.ident,res.0.8,S.Score,G2M.Score,Phase
P1TLH_AAACCTGAGCAGCCTC_1,2943,1427,P1TLH,12,0.046089,0.000349,S
P1TLH_AAACCTGTCCTCATTA_1,10897,2522,P1TLH,17,-0.000357,0.009434,G2M
P1TLH_AAACCTGTCTAAGCCA_1,1914,1018,P1TLH,12,0.012811,-0.056561,S
P1TLH_AAACGGGAGTAGGCCA_1,5574,1798,P1TLH,10,-0.011324,-0.047102,G1
P1TLH_AAACGGGGTTCGGGCT_1,3700,1417,P1TLH,2,0.057467,-0.003861,S


`clusterid_to_clustername` is used to convert integers in `res.0.8` to cell-type name

In [8]:
clusterid_to_clustername=pd.read_csv('data/extract/HumanLiver.clusterid_to_clustername.tsv',sep='\t',header=None,index_col=0)
len(clusterid_to_clustername[1].unique()),clusterid_to_clustername

(11,
                          1
 0                         
 1              Hepatocytes
 2               ab_T_cells
 3              Hepatocytes
 4              Macrophages
 5              Hepatocytes
 6              Hepatocytes
 7             Plasma_cells
 8                 NK_cells
 9               gd_T_cells
 10             Macrophages
 11                   LSECs
 12                   LSECs
 13                   LSECs
 14             Hepatocytes
 15             Hepatocytes
 16          Mature_B_cells
 17          Cholangiocytes
 18              gd_T_cells
 19         Erythroid_cells
 20  Hepatic_Stellate_Cells)

## marker info

In [9]:
clustername2markers={'Hepatocytes':['ALB','HAMP','ARG1','PCK1','AFP','BCHE'],
'LSECs':['CALCRL','CD32B','VWF'],
'Cholangiocytes':['KRT19','EPCAM','FXDY2','CLDN4','CLDN10','SOX9','MMP7','CXCL1','CFTR','TFF2','KRT7','CD24'],
'Hepatic_Stellate_Cells':['ACTA2','COL1A1','TAGLN','COL1A2','COL3A1','SPARC','RBP1','DCN','MYL9'],
'Macrophages':['CD68','MARCO'],
'ab_T_cells':['CD2','CD3D','TRAC','IL32','CD3E'],
'gd_T_cells':['NKG7','FCGR3A','HOPX','GNLY'],
'NK_cells':['GZMK','KLRF1','CCL3','CMC1'],
'Plasma_cells':['CD27','IGHG1'],
'Mature_B_cells':['MS4A1','LTB','CD52','IGHD'],
'Erythroid_cells':['HBB','SLC25A37','CA1','ALAS2']
}

In [10]:
clustername_to_markers_new={'Cholangiocytes':['KRT19','EPCAM','FXYD2','CLDN4','CLDN10','SOX9','MMP7','CXCL1','CFTR','TFF2','KRT7','CD24'],
'Mature_B_cells':['MS4A1','IGHD','CD79A','PTPRC','IGKC','CD19'],
'Hepatocytes':['ALB','HAMP','ARG1','PCK1','AFP','BCHE'],
'LSECs':['CALCRL','VWF','PECAM1','CLEC14A','EMCN'],
'Hepatic_Stellate_Cells':['ACTA2','COL1A1','TAGLN','COL1A2','COL3A1','SPARC','RBP1','DCN','MYL9'],
'Macrophages':['CD68','MARCO','FCGR3A','LYZ','PTPRC','AIF1'],
'ab_T_cells':['CD2','CD3D','TRAC','IL32','CD3E','PTPRC'],
'gd_T_cells':['NKG7','FCGR3A','HOPX','GNLY','CMC1','KLRF1','CCL3','PTPRC'],
'NK_cells':['GZMK','KLRF1','CCL3','CMC1','NKG7','PTPRC'],
'Plasma_cells':['CD27','IGHG1','IGHA1','IGHM','CD79A','PTPRC','IGKC'],
'Erythroid_cells':['HBB','SLC25A37','CA1','ALAS2']}

#for key,value in clustername2markers_new.items():
#    for i in value:
#        print(markers_db[(markers_db['official gene symbol']==i)].shape)
#markers_db[(markers_db['official gene symbol']=='CD32B') |(markers_db['nicknames'].str.contains('CD32B'))]

In [11]:
clustername_unique=list(clustername_to_markers_new.keys())
exp_data_meta_clusterid_clusteridunique=clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].apply(lambda x: clustername_unique.index(x))

In [12]:
clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].value_counts()

Hepatocytes               3501
Macrophages               1192
ab_T_cells                 961
LSECs                      844
gd_T_cells                 569
Plasma_cells               511
NK_cells                   488
Mature_B_cells             129
Cholangiocytes             119
Erythroid_cells             93
Hepatic_Stellate_Cells      37
Name: 1, dtype: int64

In [13]:
np.unique(clustername_to_markers_new.values())

array([dict_values([['KRT19', 'EPCAM', 'FXYD2', 'CLDN4', 'CLDN10', 'SOX9', 'MMP7', 'CXCL1', 'CFTR', 'TFF2', 'KRT7', 'CD24'], ['MS4A1', 'IGHD', 'CD79A', 'PTPRC', 'IGKC', 'CD19'], ['ALB', 'HAMP', 'ARG1', 'PCK1', 'AFP', 'BCHE'], ['CALCRL', 'VWF', 'PECAM1', 'CLEC14A', 'EMCN'], ['ACTA2', 'COL1A1', 'TAGLN', 'COL1A2', 'COL3A1', 'SPARC', 'RBP1', 'DCN', 'MYL9'], ['CD68', 'MARCO', 'FCGR3A', 'LYZ', 'PTPRC', 'AIF1'], ['CD2', 'CD3D', 'TRAC', 'IL32', 'CD3E', 'PTPRC'], ['NKG7', 'FCGR3A', 'HOPX', 'GNLY', 'CMC1', 'KLRF1', 'CCL3', 'PTPRC'], ['GZMK', 'KLRF1', 'CCL3', 'CMC1', 'NKG7', 'PTPRC'], ['CD27', 'IGHG1', 'IGHA1', 'IGHM', 'CD79A', 'PTPRC', 'IGKC'], ['HBB', 'SLC25A37', 'CA1', 'ALAS2']])],
      dtype=object)

In [14]:
marker_unique=np.unique([j for i in list(clustername_to_markers_new.values()) for j in i])
marker_unique_exp_data_idx=[exp_data_row.index(marker) for marker in marker_unique]

In [15]:
pi_true=np.array([np.sum(exp_data_meta_clusterid_clusteridunique==i) for i in sorted(np.unique(exp_data_meta_clusterid_clusteridunique))])/exp_data_meta_clusterid_clusteridunique.shape[0]
M_true=np.array([np.mean(exp_data[marker_unique_exp_data_idx,:][:,exp_data_meta_clusterid_clusteridunique==i],axis=1) for i in sorted(np.unique(exp_data_meta_clusterid_clusteridunique))])

In [16]:
clustername_unique
np.unique(list(map(lambda x: clustername_unique.index(x),clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1])),return_counts=True)[1],pi_true*8444

(array([ 119,  129, 3501,  844,   37, 1192,  961,  569,  488,  511,   93]),
 array([ 119.,  129., 3501.,  844.,   37., 1192.,  961.,  569.,  488.,
         511.,   93.]))

# Our

In [15]:
def safe_log_sum(logx, logy):
    if logx >= logy:
        return logx + np.log(1+np.exp(logy-logx))
    else:
        return logy + np.log(1+np.exp(logx-logy))

def safe_log_sum_vector(vec):
    # safe_log_sum    
    log_sum = vec[-1]
    for i in range(vec.shape[0]-1):
        log_sum = safe_log_sum(log_sum, vec[i])
    
    return log_sum

In [201]:
cell_size_factor=pd.read_csv('data/analysis/size_factor_cluster.tsv',sep='\t',header=None)[0].values#.reshape(-1,1)

In [222]:
num_type=11

Y=exp_data[marker_unique_exp_data_idx].transpose().astype(float)/cell_size_factor.reshape(-1,1)
Y_gammaln=sc.gammaln(Y+1)
Y.shape,Y,Y_gammaln

marker_onehot=np.array([np.sum(np.eye(len(marker_unique))[[marker_unique.tolist().index(marker) for marker in value]],axis=0) for key,value in clustername_to_markers_new.items()])
marker_onehot.shape

pi=np.ones(shape=num_type) / num_type

In [223]:
exp_data_col_patient=pd.Series(exp_data_col).str.slice(start=1,stop=2).astype(int)
x_data_covariate=np.eye(len(exp_data_col_patient.unique()))[exp_data_col_patient.values-1]
x_data_intercept=np.array([np.ones(Y.shape[0])]).transpose()
x_data_null=np.concatenate([x_data_intercept,x_data_covariate[:,:-1]],axis=1)
x_data_null.shape

beta=np.zeros((x_data_null.shape[1],Y.shape[1]))

In [224]:
mu_type_init=np.where(marker_onehot==0,np.random.uniform(5,10,size=(marker_onehot.shape[0],marker_onehot.shape[1])),0)+\
np.where(marker_onehot!=0,np.random.normal(np.mean(np.max(Y,axis=0)),np.mean(np.max(Y,axis=0))/50,size=(marker_onehot.shape[0],marker_onehot.shape[1])),0)
mu_type_init=(mu_type_init/np.mean(mu_type_init,axis=1).reshape(-1,1)*np.mean(mu_type_init))

In [225]:
np.min(mu_type_init)

2.8909702987821544

In [226]:
mu_type=mu_type_init

In [227]:
np.max(np.mean(Y,axis=0))

118.54291720410605

In [228]:
cell_size_factor

array([1.0309219 , 2.38041632, 0.72895119, ..., 1.45360587, 0.51372201,
       0.40260243])

In [229]:
def E_step(Y, mu, pi):    
    log_gamma=np.zeros(shape=(Y.shape[0], mu_type.shape[0]))
    for i in range(Y.shape[0]): # prange if needed
        for t in range(mu_type.shape[0]):
            for f in range(Y.shape[1]): # likelihood using sc special funcs
                LL=sc.xlogy(Y[i,f],mu[i,t,f]) - Y_gammaln[i,f] -(mu[i,t,f])
                log_gamma[i,t] += LL
            log_gamma[i,t] += np.log(pi[t]) 
    print(np.min(log_gamma),np.max(log_gamma))
    for i in range(Y.shape[0]):
        log_gamma[i,:] = log_gamma[i,:] - safe_log_sum_vector(log_gamma[i,:])

    return np.exp(log_gamma),log_gamma
#3E_step(Y,mu,pi)
#pi
#np.sum(gamma, axis=0)
#gamma
def M_step(Y,cell_size_factor,gamma,mu_type,mu_null,x_data_null):
    
    mu_type_new = np.zeros(shape=(gamma.shape[1],Y.shape[1])) # type feature
    for i in range(Y.shape[0]):
        for t in range(gamma.shape[1]):
            for f in range(Y.shape[1]):
                mu_type_new[t,f]+=gamma[i][t]*(Y[i][f]-mu_null[i,f])
    mu_type_new=mu_type_new/np.sum(gamma,axis=0).reshape(-1,1)
    #mu_type=np.where(mu_type>0,mu_type,0.1)
    
    
    mu_null_new=np.zeros(shape=(mu_null.shape[0],mu_null.shape[1])) #  cell  feature
    for i in range(Y.shape[0]):
        for t in range(gamma.shape[1]):
            for f in range(Y.shape[1]):
                mu_null_new[i,f]+=gamma[i][t]*(Y[i][f]-mu_type[t,f]) 
    #mu_null=mu_null/np.sum(gamma,axis=0).reshape(-1,1)           
    #mu_null=np.where(mu_null>0,mu_null,0.1)
           
    reg=LinearRegression(fit_intercept=False).fit(x_data_null, mu_null_new)
    beta=reg.coef_.T
        
    #mu_null=mu_null/np.sum(gamma,axis=1).reshape(-1,1)
    
    return mu_type_new,mu_null_new,beta


In [230]:
#mu_null/np.sum(gamma,axis=1).reshape(-1,1)     
mu_null.shape,gamma.shape

((8444, 63), (8444, 11))

In [231]:
#np.sum(gamma,axis=0).reshape(-1,1)     
np.sum(gamma,axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [232]:
mu_type.max(),np.max(Y)

(2279.498146066211, 60680.554482129875)

In [233]:
((mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)<0).sum()

133757

In [234]:
param_list=[]

In [236]:
for i in range(20):
    print(np.max(mu_type,axis=1))
    print(pi)

    mu_null=np.dot(x_data_null,beta)
    mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
    
    #mu*=cell_size_factor.reshape(-1,1)
    mu=np.where(mu>0,mu,0)+np.where(mu<0,5,0)


    gamma,_ = E_step(Y,mu,pi)
    pi = np.sum(gamma, axis=0) / Y.shape[0]
    mu_type,mu_null,beta=M_step(Y,cell_size_factor,gamma,mu_type,mu_null,x_data_null)    
    print((pi*8444).astype(int))
    print(np.sum(np.argmax(gamma,axis=1)==list(map(lambda x: clustername_unique.index(x),clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1]))))
    param_list.append([pi,mu,mu_type,mu_null,beta])

[  780.00399401 12554.56165686   779.31032379   779.45805516
  1273.88598317  1452.58317724  4948.43190008   888.05616614
   779.53859367  3673.9844685  36490.46764577]
[0.40629277 0.00450024 0.53442065 0.00259192 0.00450024 0.00236855
 0.0039081  0.01184278 0.02164014 0.00698721 0.00094742]
-397567.22203092003 -8222.810976517256
[3234   38 4728   20   29   22   31   86  191   55    8]
388
[  779.45875897 12548.34918913   569.89726272   660.42532705
  1467.38744509  1556.18964257  5086.41929452  1027.18142552
   605.50205876  3890.69335146 36482.57913583]
[0.38309657 0.00450024 0.559941   0.00236855 0.00343439 0.0026054
 0.00367125 0.01029846 0.02262323 0.0065135  0.00094742]
-438046.8615805007 -277.57622609530483
[3317   37 4638   21   38   21   33  100  170   59    8]
353
[  779.98661068 12672.44376957   779.31176112   779.45201365
  1250.83677313  1505.27688412  5123.68825845   887.13849067
   779.52270044  3671.19259872 36493.1732752 ]
[0.39287313 0.00438181 0.54931792 0.00248697 0

In [221]:
M_true[0],mu_type[0],

(array([1.68067227e-02, 0.00000000e+00, 1.09243697e-01, 2.52100840e-02,
        8.16722689e+01, 1.09243697e-01, 5.88235294e-02, 1.68067227e-02,
        1.68067227e-02, 2.52100840e-02, 0.00000000e+00, 0.00000000e+00,
        6.53781513e+00, 0.00000000e+00, 4.20168067e-02, 0.00000000e+00,
        7.56302521e-02, 8.40336134e-03, 3.36134454e-01, 1.94117647e+00,
        1.69747899e+00, 8.40336134e-03, 1.84873950e-01, 1.68067227e-02,
        0.00000000e+00, 0.00000000e+00, 2.45378151e+00, 0.00000000e+00,
        0.00000000e+00, 1.42857143e+00, 4.20168067e-02, 9.45378151e+00,
        5.88235294e-02, 8.40336134e-03, 1.76470588e-01, 1.25210084e+00,
        1.68067227e-02, 1.42857143e-01, 0.00000000e+00, 1.26050420e-01,
        2.18487395e-01, 9.66386555e-01, 1.30252101e+00, 3.36134454e-02,
        3.57983193e+00, 4.24369748e+00, 4.26050420e+00, 8.40336134e-03,
        5.88235294e-01, 0.00000000e+00, 1.09243697e-01, 1.68067227e-01,
        3.52941176e-01, 8.40336134e-03, 3.36134454e-02, 4.537815

In [ ]:

print(np.max(mu_type,axis=1))
print(pi)

mu_null=cell_size_factor.reshape(-1,1)*np.dot(x_data_null,beta)
#mu_null=np.where(mu_null>-50,mu_null,-50)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0)+np.where(mu<0,5,0)


gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    
print((pi*8444).astype(int))
print(np.sum(np.argmax(gamma,axis=1)==list(map(lambda x: clustername_unique.index(x),clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1]))))
param_list.append([pi,mu,mu_type,mu_null,beta])

In [124]:

print(np.max(mu_type,axis=1))
print(pi)

mu_null=cell_size_factor.reshape(-1,1)*np.dot(x_data_null,beta)
#mu_null=np.where(mu_null>-50,mu_null,-50)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0)+np.where(mu<0,5,0)


gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    
print((pi*8444).astype(int))
print(np.sum(np.argmax(gamma,axis=1)==list(map(lambda x: clustername_unique.index(x),clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1]))))
param_list.append([pi,mu,mu_type,mu_null,beta])

[  67.01592848  145.15954379  219.26619855   26.55876291   87.2427411
   30.4219869    10.25649865   17.01405711   11.75997344 1260.58419348
 2448.30661958]
[0.00416676 0.02677531 0.41802871 0.19461507 0.01616555 0.1117985
 0.07746171 0.05197071 0.04833735 0.03800861 0.01267172]
-223817.1471593248 -57.94327859977274
[  38  791 2827 2899   51  330  856  161  166  221   97]
4736


In [125]:

print(np.max(mu_type,axis=1))
print(pi)

mu_null=cell_size_factor.reshape(-1,1)*np.dot(x_data_null,beta)
#mu_null=np.where(mu_null>-50,mu_null,-50)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0)+np.where(mu<0,5,0)


gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    
print((pi*8444).astype(int))
print(np.sum(np.argmax(gamma,axis=1)==list(map(lambda x: clustername_unique.index(x),clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1]))))
param_list.append([pi,mu,mu_type,mu_null,beta])

[  94.39847387  174.35596969  274.05563459   10.65850142   43.10595464
   16.24652764    9.53354247   25.30682085   17.38396417 1754.99271944
 2657.70910478]
[0.00457098 0.09374233 0.33489868 0.34341241 0.00610872 0.03919386
 0.10146368 0.0190671  0.01976406 0.02629072 0.01148745]
-192629.0228947786 -41.50085871086665
[  46  548 2792 2894   39  358 1148   89  233  194   98]
4502


In [109]:
np.unique(list(map(lambda x: clustername_unique.index(x),clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1])),return_counts=True)[1]

array([ 119,  129, 3501,  844,   37, 1192,  961,  569,  488,  511,   93])

In [228]:
np.sum(beta,axis=0)

array([ 7.60961147e-01, -7.66725017e-01, -6.28592719e-01,  1.89757646e+00,
       -4.24716959e+02, -7.98621653e+00, -3.30476665e+00,  2.61620738e+01,
       -4.47660406e-01, -2.62412731e+00, -1.59678590e-01,  1.56422602e+00,
       -1.00774111e+00, -1.52030203e+00,  1.50478727e+00,  3.09901655e+00,
       -2.66748443e-01, -1.81064702e+00, -3.19695712e-02, -2.65737758e-01,
       -1.55502514e-01, -1.17139006e+00,  1.26087851e+01,  5.87401261e-01,
        4.99142394e-01,  1.73387132e-01, -4.80371358e-01,  3.44623235e-01,
       -4.73516388e-02, -1.29997791e-01,  1.18062427e+00, -1.13202742e+00,
        1.32751960e+01,  9.38052697e-01, -3.09163123e+01,  2.61840081e+02,
        1.37065933e+00, -1.12956280e+02, -2.43066359e-01, -1.43096516e+02,
       -6.22441187e+01, -5.44777275e+02, -1.82822902e+00,  9.74390185e-01,
        1.67524294e-01, -3.62493298e-01, -9.18007308e+00,  1.34927412e+00,
       -4.21253790e-02, -7.33423046e-01,  8.32361641e-02,  2.30135285e+01,
       -9.18491577e+00, -

In [135]:
i=1
a=1
np.max(param_list[a][i]),np.min(param_list[a][i]),np.mean(param_list[a][i]),'-----',np.max(M_true),np.min(M_true),np.mean(M_true)

(1485.1284449737188,
 9.149994834817353e-37,
 6.537284183792937,
 '-----',
 2627.94623655914,
 0.0,
 6.811061848758416)

In [128]:
i=1
param_list[0][1][0]#,param_list[3][i]

array([-2.09896468, -2.15768241, -2.09895425, -2.09896382,  9.94010302,
       -1.9815158 , -2.01088686, -2.12832341, -2.12831994, -2.1283163 ,
       -2.1576824 , -2.15768241,  9.62742936, -2.15768241, -2.15768236,
       -2.15768236, -2.04023108, -2.12832321, -1.57056128,  0.92488512,
        1.21951904, -2.1266231 , -1.89344661, -2.06960581, -2.15768239,
       -2.15768241,  0.11773849, -1.95217035, -2.15766733, -0.21912543,
       -2.12832354, 12.52006441, -2.12832348, -2.15768241, -1.98151355,
       -0.36480989, -2.12832354, -1.95319618, -2.15768217, -2.06958089,
       -1.95321154, -1.21816933, -1.01473372, -2.12832354,  8.67750257,
        2.66122803, 10.40794249, -2.12832177, -0.92773646, -2.15768241,
       -1.98152842, -2.01192773, -1.15948047, -2.12832126, -2.09896439,
       -1.92286698, -1.3943968 , -0.72031575, -2.09892347, -2.12832281,
       40.08972519, -2.15768241, -2.15768205])

In [153]:
for i in range(63):
    print(param_list[1][1][0][i].astype(int),mu_type_init[0][i].astype(int))

-2 3
-1 5
-1 3
-1 4
7 4
-1 3
-1 3
-1 4
-1 4
-1 4
-1 3
-2 3
9 581
-1 3
-2 3
-2 5
-1 5
-1 3
-1 601
1 583
0 562
-1 4
-2 5
-1 5
-2 3
-1 3
1 566
-1 4
-1 4
0 583
-2 4
11 588
-2 3
-2 3
-1 4
22 3
-2 3
1 3
-1 4
9 5
0 5
4 5
0 5
-1 3
7 595
2 609
10 4
-2 5
0 577
-1 5
-1 4
-3 5
-1 4
-1 4
-2 4
-1 4
-1 3
0 588
-2 3
-1 3
33 591
-2 5
-1 4


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0])

In [28]:

print(np.max(mu_type,axis=1))
print(pi)

mu_null=cell_size_factor.reshape(-1,1)+np.dot(x_data_null,beta)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0.1)


gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    
print((pi*8444).astype(int))

[  31.12882289   82.85407948  211.89656871   80.39016246   44.42824689
   33.72774873   22.38548037   17.81454516   23.43162545 1225.59050147
 2494.56890807]
[0.00608436 0.01119158 0.35297556 0.14253619 0.00772794 0.12812218
 0.1659153  0.05574914 0.07498457 0.04227833 0.01243486]
-175020.11918723217 -13.956508958914547
[ 104  120 2783  900   71  878 1925  579  657  319  103]


In [29]:

print(np.max(mu_type,axis=1))
print(pi)

mu_null=cell_size_factor.reshape(-1,1)+np.dot(x_data_null,beta)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0.1)


gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    
print((pi*8444).astype(int))

[  48.58310342  202.39084795  225.25660223  137.04688217   59.50634538
   53.39068676   22.44376804   17.94991637   19.25324374 1365.46713383
 2533.85953592]
[0.01235106 0.01427113 0.32967901 0.10664645 0.00845194 0.10403465
 0.22808279 0.0686739  0.07783246 0.03777861 0.01219801]
-177791.2841974742 -17.151687444304
[ 243  141 2587  983   60  486 2067  614  858  297  102]


In [30]:

print(np.max(mu_type,axis=1))
print(pi)

mu_null=cell_size_factor.reshape(-1,1)+np.dot(x_data_null,beta)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0.1)


gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    
print((pi*8444).astype(int))

[  74.05464734  289.12719632  246.66740001  174.03390935   89.3753357
  106.76916817   23.52985975   17.93064193   18.22124227 1465.14625428
 2555.16038816]
[0.02887429 0.01676547 0.30642086 0.11650331 0.00712817 0.05762977
 0.24487613 0.07279474 0.10165766 0.03527001 0.01207958]
-181048.84051002824 -17.144718015887335
[ 288  163 2445 1084   42  268 1818  758 1195  277  102]


In [31]:

print(np.max(mu_type,axis=1))
print(pi)

mu_null=cell_size_factor.reshape(-1,1)+np.dot(x_data_null,beta)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0.1)


gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    
print((pi*8444).astype(int))

[ 102.59118377  337.6503708   264.99091593  210.75887325  129.18223525
  204.38956951   24.62880706   20.93558337   16.46666055 1583.96304898
 2555.41382533]
[0.03412718 0.01937107 0.28960704 0.12841501 0.00508884 0.03182805
 0.21532079 0.08980242 0.14154298 0.03281705 0.01207958]
-178568.3397025616 -15.991467648629534
[ 268  159 2317 1114   28  343 1266 1362 1215  265  102]


array([ 119,  129, 3501,  844,   37, 1192,  961,  569,  488,  511,   93])

In [158]:

print(np.max(mu_type,axis=1))
print(pi)
      
mu_null=cell_size_factor.reshape(-1,1)+np.dot(x_data_null,beta)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0)

gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    

[  38.27693169  102.46117534  176.79433641    2.98713764   13.49770148
    8.26891408    4.67045353   13.36116519    8.93255333  667.8713617
 1502.09786507]
[0.00421551 0.00883777 0.47173446 0.0450563  0.02574351 0.12164267
 0.07649449 0.07155815 0.0864888  0.06747553 0.02075282]
-inf -23.453833832015743


/home/ch6845/tools/miniconda3/envs/singlecell/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ch6845/tools/miniconda3/envs/singlecell/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [140]:

print(np.max(mu_type,axis=1))
print(pi)
      
mu_null=cell_size_factor.reshape(-1,1)+np.dot(x_data_null,beta)
mu=(mu_null.reshape(mu_null.shape[0],1,mu_null.shape[1])+mu_type)
mu=np.where(mu>0,mu,0)

gamma,_ = E_step(Y,mu,pi)
pi = np.sum(gamma, axis=0) / Y.shape[0]
mu_type,beta=M_step(Y,cell_size_factor,gamma,mu_null,x_data_null)    

[  39.80087436  109.56413326  176.08164055    3.10590046   13.35522849
    8.3458189     4.69519508   13.58351272    8.8313321   664.28711958
 1511.27232724]
[0.00406268 0.00821568 0.47373624 0.04397894 0.02550802 0.12035187
 0.07672265 0.07040989 0.08854037 0.06784707 0.0206266 ]
[[0.34873187 0.60064804 1.46714776 ... 0.73513704 0.87207477 0.46418287]
 [1.69822629 1.95014247 2.81664218 ... 2.08463146 2.22156919 1.8136773 ]
 [0.04676116 0.29867733 1.16517705 ... 0.43316632 0.57010406 0.16221216]
 ...
 [1.45360587 1.45360587 1.45360587 ... 1.45360587 1.45360587 1.45360587]
 [0.51372201 0.51372201 0.51372201 ... 0.51372201 0.51372201 0.51372201]
 [0.40260243 0.40260243 0.40260243 ... 0.40260243 0.40260243 0.40260243]] [[[ 0.34873187  0.60064804  1.46714776 ... 40.53601139  0.87207477
    0.46418287]
  [ 0.34873187  0.60064804  1.46714776 ...  0.73513704  0.87207477
    0.46418287]
  [ 0.34873187  0.60064804  1.46714776 ...  0.73513704  0.87207477
    0.46418287]
  ...
  [ 0.34873187  0.6

/home/ch6845/tools/miniconda3/envs/singlecell/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ch6845/tools/miniconda3/envs/singlecell/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [80]:
temp_null=M_step2(Y,cell_size_factor,gamma,mu_null,x_data_null)  

In [82]:
gamma,mu_null,temp_null

(array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 array([[0.33333828, 0.6733144 , 1.42802075, ..., 0.81935536, 0.97437863,
         0.43405014],
        [1.6828327 , 2.02280882, 2.77751517, ..., 2.16884978, 2.32387305,
         1.78354456],
        [0.03136756, 0.37134369, 1.12605003, ..., 0.51738465, 0.67240792,
         0.13207942],
        ...,
        [1.45360587, 1.45360587, 1.45360587, ..., 1.45360587, 1.45360587,
         1.45360587],
        [0.51372201, 0.51372201, 0.51372201, ..., 0.51372201, 0.51372201,
         0.51372201],
        [0.40260243, 0.40260243, 0.40260243, ..., 0.40260243, 0.40260243,
         0.40260243]]),
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, 

In [51]:
mu_null,mu

(array([[1.0309219 , 1.0309219 , 1.0309219 , ..., 1.0309219 , 1.0309219 ,
         1.0309219 ],
        [2.38041632, 2.38041632, 2.38041632, ..., 2.38041632, 2.38041632,
         2.38041632],
        [0.72895119, 0.72895119, 0.72895119, ..., 0.72895119, 0.72895119,
         0.72895119],
        ...,
        [1.45360587, 1.45360587, 1.45360587, ..., 1.45360587, 1.45360587,
         1.45360587],
        [0.51372201, 0.51372201, 0.51372201, ..., 0.51372201, 0.51372201,
         0.51372201],
        [0.40260243, 0.40260243, 0.40260243, ..., 0.40260243, 0.40260243,
         0.40260243]]),
 array([[[   6.76532477,    5.55684432,    4.45142528, ...,
           596.87243118,    6.26628776,    4.73893481],
         [   9.26590104,    8.14480269,   11.47122076, ...,
             9.04538071,   11.244965  ,    7.35100509],
         [   9.06345624, 1108.73509465,    9.29731481, ...,
             7.01066575,    9.70611684,    6.65477778],
         ...,
         [   7.64176888,    9.59826799,    8.52

In [53]:
mu_null,mu

(array([[1.0309219 , 1.0309219 , 1.0309219 , ..., 1.0309219 , 1.0309219 ,
         1.0309219 ],
        [2.38041632, 2.38041632, 2.38041632, ..., 2.38041632, 2.38041632,
         2.38041632],
        [0.72895119, 0.72895119, 0.72895119, ..., 0.72895119, 0.72895119,
         0.72895119],
        ...,
        [1.45360587, 1.45360587, 1.45360587, ..., 1.45360587, 1.45360587,
         1.45360587],
        [0.51372201, 0.51372201, 0.51372201, ..., 0.51372201, 0.51372201,
         0.51372201],
        [0.40260243, 0.40260243, 0.40260243, ..., 0.40260243, 0.40260243,
         0.40260243]]),
 array([[[   6.76532477,    5.55684432,    4.45142528, ...,
           596.87243118,    6.26628776,    4.73893481],
         [   9.26590104,    8.14480269,   11.47122076, ...,
             9.04538071,   11.244965  ,    7.35100509],
         [   9.06345624, 1108.73509465,    9.29731481, ...,
             7.01066575,    9.70611684,    6.65477778],
         ...,
         [   7.64176888,    9.59826799,    8.52

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [175]:
mu_type.sum(),mu_type2.sum(),mu_null.sum(),mu_null2.sum()

(80049.74603258204, 5996.595018158268, 531971.9999999999, 531971.9999999991)

In [176]:
mu_null2.shape

(8444, 63)

In [183]:
x_data_null.shape,beta.shape,

((8444, 5), (5, 63))

In [21]:
beta.shape

(5, 63)

In [20]:
reg.coef_.shape

NameError: name 'reg' is not defined

In [181]:
#family=sm.families.Poisson(link=sm.families.links.log())
family=sm.families.Gaussian(link=sm.families.links.identity())

In [ ]:
ValueError: operands could not be broadcast together with shapes

In [185]:
model=sm.GLM(mu_null2,x_data_null,family=family)
model_result=model.fit()

ValueError: operands could not be broadcast together with shapes (8444,63) (8444,) 

In [156]:
mu_type.shape,mu_type2.shape
mu_type2/np.sum(gamma,axis=0).reshape(-1,1)

array([[-2.63161466e+00, -2.68718705e+00, -2.63191634e+00,
        -2.63161604e+00,  3.06429631e+01, -2.29818031e+00,
        -2.63161466e+00, -2.68718705e+00, -2.63191634e+00,
        -2.68718705e+00, -2.68718705e+00, -2.68718705e+00,
         1.08670217e+01, -2.68718705e+00, -2.68718705e+00,
        -2.68718705e+00, -2.57604227e+00, -2.68718705e+00,
        -2.02092308e+00,  1.59007431e+00,  2.25211345e+00,
        -2.63161604e+00, -2.13146451e+00, -2.63161466e+00,
        -2.68718705e+00, -2.68718705e+00,  1.63988737e+00,
        -2.68718705e+00, -2.68718705e+00,  3.55570946e-02,
        -2.68718705e+00,  1.90358478e+01, -2.63161466e+00,
        -2.68718705e+00, -2.40932509e+00,  2.05402545e+01,
        -2.63191634e+00, -2.63191634e+00, -2.68718705e+00,
        -2.57634395e+00, -2.46519917e+00, -1.68779178e+00,
        -1.07619516e+00, -2.68718705e+00,  9.70333773e+00,
         1.36686598e+00,  1.22608690e+01, -2.68718705e+00,
        -1.29818168e+00, -2.68718705e+00, -2.63161466e+0

In [ ]:
Y,mu_null,mu_type2,mu_null2

In [ ]:
mu_type[0],mu_type2[0]

In [ ]:
mu_type.shape,mu_type2.shape,mu_null.shape

In [ ]:
mu_null=np.zeros(mu_null.shape[0],mu_null.shape[1])

for i in range(Y.shape[0]):
    for t in range(gamma.shape[1]):
        for f in range(Y.shape[1]):
            mu_null[i,f]=+=gamma[i][t]*(Y[i][f]-mu_type[i,f])

In [ ]:
def M_step(Y,gamma):
    
    mu = np.zeros(shape=(Y.shape[0],gamma.shape[1],Y.shape[1]))
    
    
    for t in range(gamma.shape[1]):
        for f in range(Y.shape[1]):
            #print(np.sum(gamma[:,t]))
            mu[t,f] = np.dot(gamma[:,t],Y[:,f])/np.sum(gamma[:,t])
    
    return mu


In [ ]:
mu = np.zeros(shape=(Y.shape[0],gamma.shape[1],Y.shape[1]))

In [ ]:
mu.shape

In [ ]:
gamma.shape

In [ ]:
gamma,_ = E_step(Y,mu_type,pi,beta,x_data_null)

In [ ]:
M_step(Y,gamma)

In [ ]:
x_data_intercept=np.array([np.ones(exp_data.shape[1])]).transpose()
x_data_intercept.shape

In [ ]:
x_data_null=np.concatenate([x_data_intercept,x_data_covariate[:,:-1]],axis=1)

In [ ]:
family=sm.families.Poisson(link=sm.families.links.log())

In [ ]:
def E_step(Y,mu_type):
    log_gamma=np.zeros(shape=(num_sample, num_type))
    for i in range(num_sample): # prange if needed
        for t in range(num_type):
            for f in range(num_feature): # likelihood using sc special funcs
                #LL=sc.xlogy(Y[i,f],mu_type[t,f]+mu_inflation[i,f]) - Y_gammaln[i,f] -(mu_type[t,f]+mu_inflation[i,f])
                LL=sc.xlogy(Y[i,f],mu_type[t,f]+mu_inflation[i,f]) - Y_gammaln[i,f] -(mu_type[t,f])
                log_gamma[i,t] += LL
            log_gamma[i,t] += np.log(propor[t]) 
                #print(LL)
                #print(Y[i,f])
                #print(sc.xlogy(Y[i,f],mu_type[t,f]),Y_gammaln[i,f],mu_type[t,f])
                #print(log_gamma[i,t])
                #break
            #log_gamma[i,t] += np.log(pi[t])
        #print(log_gamma[i,:])
    #return 1,log_gamma
    for i in range(num_sample):
        log_gamma[i,:] = log_gamma[i,:] - safe_log_sum_vector(log_gamma[i,:])
        #break
        
    
    return np.exp(log_gamma),log_gamma


def M_step(Y,gamma):
    mu_type = np.zeros(shape=(num_type,num_feature))
    for t in range(num_type):
        for f in range(num_feature):
            #print(np.sum(gamma[:,t]))
            mu_type[t,f] = np.dot(gamma[:,t],Y[:,f])/np.sum(gamma[:,t])
    
    return mu_type

In [ ]:
y_data.shape,x_data_null.shape

In [ ]:
y_data.shape

In [ ]:
max(cell_size_factor)

In [ ]:
family=sm.families.Poisson(link=sm.families.links.log())
#family=sm.families.Poisson(link=sm.families.links.identity())

In [ ]:
coef_list=[] #intercept, covariate 4
ll_list=[]

for exp_data_row_idx in range(exp_data.shape[0]):
    if exp_data_row_idx%100==0:
        #sys.stdout.write('\r%0.2f%%' % (100.0 * (row_idx/weight_total.shape[0])))
        sys.stdout.write("\r{:f}%%".format(100*exp_data_row_idx/exp_data.shape[0]))
        sys.stdout.flush()     
    
    y_data=exp_data[exp_data_row_idx,:]/cell_size_factor
    model=sm.GLM(y_data,x_data_null,family=family)
    model_result=model.fit()
    
    coef_list.append(model_result.params)
    ll_list.append(model_result.llf)

#model_result.summary()
#ll_list.append(model_result.llf)
#model_result.llf,model_result.llnull,model_result.null_deviance,model_result.null
#model=sm.GLM([100]*5000+[1000]*5000,[[1,0]]*(5000-1)+[[1,1]]*(5000+1),family=family)
#model_result=model.fit()
#model_result.summary()

In [ ]:
coef_matrix=np.array(coef_list)

In [ ]:
exp_data[marker_unique_exp_data_idx,:].astype(float)/cell_size_factor

In [ ]:
exp_data
cell_size_factor
x_data_covariate
clustername_to_markers_new

num_type=11

In [ ]:
marker_unique=np.unique([j for i in list(clustername_to_markers_new.values()) for j in i])
marker_unique_exp_data_idx=[exp_data_row.index(marker) for marker in marker_unique]

In [ ]:
marker_onehot=np.array([np.sum(np.eye(len(marker_unique))[[marker_unique.tolist().index(marker) for marker in value]],axis=0) for key,value in clustername_to_markers_new.items()])
marker_onehot.shape

In [ ]:
#np.eye(len(marker_unique))

In [ ]:
#[marker_unique.index(marker) for marker in value]

In [ ]:
#np.max(exp_data),np.min(exp_data)
np.sum(exp_data==0),np.sum(exp_data==1),np.sum(exp_data>1)
plt.hist(np.log10(exp_data[exp_data>1].reshape(-1)),bins=50)

## variable init

In [ ]:
#np.where(marker_onehot==0,np.random.uniform(5,10,size=(marker_onehot.shape[0],marker_onehot.shape[1])),0)+\
#np.where(marker_onehot!=0,np.random.uniform(500,600,size=(marker_onehot.shape[0],marker_onehot.shape[1])),0)

#(mu_type.T/np.sum(mu_type,axis=1)).T
#np.median(exp_data[exp_data>1])
for i in range(Y.shape[1]):
    print("{:.3f}  {:.3f}  {:.3f}".format(np.max(Y,axis=0)[i],np.std(Y,axis=0)[i],np.max(Y,axis=0)[i]/np.std(Y,axis=0)[i]))
    
    

In [ ]:
np.sum(mu_type,axis=1),np.mean(mu_type,axis=1)

In [ ]:
mu_type=np.where(marker_onehot==0,np.random.uniform(5,10,size=(marker_onehot.shape[0],marker_onehot.shape[1])),0)+\
np.where(marker_onehot!=0,np.random.normal(np.mean(np.max(Y,axis=0)),np.mean(np.max(Y,axis=0))/50,size=(marker_onehot.shape[0],marker_onehot.shape[1])),0)
#np.concatenate([x_data_intercept,x_data_covariate[:,:-1]].T
mu_type,mu_type.shape

In [ ]:
mu_inflation=np.exp(np.matmul(coef_matrix,x_data_null.T))[marker_unique_exp_data_idx].transpose().astype(float)
mu_inflation.shape#type feature

In [ ]:
Y=exp_data[marker_unique_exp_data_idx].transpose().astype(float)
Y_gammaln=sc.gammaln(Y+1)
Y.shape,Y,Y_gammaln

In [ ]:
#coef_matrix[marker_unique_exp_data_idx]

In [ ]:
gamma=np.ones(shape=(Y.shape[0], num_type)) / num_type
gamma.shape

In [ ]:
#coef_matrix
#Y=(exp_data/cell_size_factor-np.matmul(coef_matrix,x_data_covariate.T)[marker_unique_exp_data_idx]).transpose().astype(float)
#(exp_data/cell_size_factor-np.exp(np.matmul(coef_matrix,x_data_null.T)))[marker_unique_exp_data_idx].transpose().astype(float)
pi=np.ones(shape=num_type) / num_type
pi.shape

## Run iter

In [ ]:
mu_type.shape,mu_inflation.shape,Y.shape


In [ ]:
gamma

In [ ]:
mu_inflation.shape

In [ ]:
def E_step(Y,mu_type):
    log_gamma=np.zeros(shape=(num_sample, num_type))
    for i in range(num_sample): # prange if needed
        for t in range(num_type):
            for f in range(num_feature): # likelihood using sc special funcs
                #LL=sc.xlogy(Y[i,f],mu_type[t,f]+mu_inflation[i,f]) - Y_gammaln[i,f] -(mu_type[t,f]+mu_inflation[i,f])
                LL= -(mu_type[t,f])
                log_gamma[i,t] += LL
                #print(LL)
                #print(Y[i,f])
                #print(sc.xlogy(Y[i,f],mu_type[t,f]),Y_gammaln[i,f],mu_type[t,f])
                #print(log_gamma[i,t])
                #break
            #log_gamma[i,t] += np.log(pi[t])
        #print(log_gamma[i,:])
    return 1,log_gamma
    for i in range(num_sample):
        log_gamma[i,:] = log_gamma[i,:] - safe_log_sum_vector(log_gamma[i,:])
        #break
        
    
    return np.exp(log_gamma),log_gamma

gamma,_=E_step(Y,mu_type)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 119,  129, 3501,  844,   37, 1192,  961,  569,  488,  511,   93]))

In [180]:
clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].value_counts()#.loc[list(clustername_to_markers_new.keys())]#[clustername_to_markers_new.keys()]

Hepatocytes               3501
Macrophages               1192
ab_T_cells                 961
LSECs                      844
gd_T_cells                 569
Plasma_cells               511
NK_cells                   488
Mature_B_cells             129
Cholangiocytes             119
Erythroid_cells             93
Hepatic_Stellate_Cells      37
Name: 1, dtype: int64

In [ ]:
np.sum(gamma,axis=0)

In [ ]:
list(clustername_to_markers_new.keys())

In [ ]:
#Y
np.mean(Y[clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values=='LSECs']),np.mean(Y[clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values=='Hepatocytes']),np.mean(Y[clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values=='Macrophages'])

In [ ]:
print(np.sum(_[:,0]))
print(np.sum(_[:,1]))
print(np.sum(_[:,2]))
print(np.sum(_[:,3]))

In [ ]:
gamma

In [ ]:
#_[:,0],_[:,1],_[:,2]
_

In [ ]:
def M_step(Y,gamma):
    mu_type = np.zeros(shape=(num_type,num_feature))
    for t in range(num_type):
        for f in range(num_feature):
            #print(np.sum(gamma[:,t]))
            mu_type[t,f] = np.dot(gamma[:,t],Y[:,f])/np.sum(gamma[:,t])
    
    return mu_type



mu_type2 = M_step(Y,gamma)
mu_type2

In [ ]:
num_sample=Y.shape[0]
num_feature=Y.shape[1]
num_type=num_type

def safe_log_sum(logx, logy):
    if logx >= logy:
        return logx + np.log(1+np.exp(logy-logx))
    else:
        return logy + np.log(1+np.exp(logx-logy))

def safe_log_sum_vector(vec):
    # safe_log_sum    
    log_sum = vec[-1]
    for i in range(vec.shape[0]-1):
        log_sum = safe_log_sum(log_sum, vec[i])
    
    return log_sum


def E_step(Y,mu_type):
    log_gamma=np.zeros(shape=(num_sample, num_type))
    for i in range(num_sample): # prange if needed
        for t in range(num_type):
            for f in range(num_feature): # likelihood using sc special funcs
                LL=sc.xlogy(Y[i,f],mu_type[t,f]) - Y_gammaln[i,f] - mu_type[t,f]
                log_gamma[i,t] += LL
                #print(LL)
                #print(Y[i,f])
                #print(sc.xlogy(Y[i,f],mu_type[t,f]),Y_gammaln[i,f],mu_type[t,f])
                #print(log_gamma[i,t])
                #break
            #log_gamma[i,t] += np.log(pi[t])
        #print(log_gamma[i,:])
        log_gamma[i,:] = log_gamma[i,:] - safe_log_sum_vector(log_gamma[i,:])
        #break
        
    
    return np.exp(log_gamma),log_gamma


def M_step(Y,gamma):
    mu_type = np.zeros(shape=(num_type,num_feature))
    for t in range(num_type):
        for f in range(num_feature):
             mu_type[t,f] = np.dot(gamma[:,t],Y[:,f])/np.sum(gamma[:,t])
    
    return mu_type

def cal_LL():
    pass
    
    
# iteration
for iter_idx in range(10):
    # expectation step: update gamma
    gamma,log_gamma = E_step(Y,mu_type)
    
    mu_type = M_step(Y,gamma)
    break
    
    # maximization step
    #pi = np.sum(gamma, axis=0) / N.shape[0]
    #pi=np.sum(gamma, axis=0) / num_sample
    #M = max_M(N, gamma) 


In [ ]:
gamma#[2]

In [ ]:
gamma=np.ones(shape=(N.shape[0], num_type)) / num_type

In [ ]:
gamma = np.ones(shape=(N.shape[0], n_type)) / n_type

In [ ]:
coef_matrix.shape,x_data_covariate.shape

In [ ]:
np.dot(coef_matrix,x_data_covariate.T)==np.matmul(coef_matrix,x_data_covariate.T)

In [ ]:
np.exp(np.matmul(coef_matrix,x_data_covariate.T)[marker_unique_exp_data_idx])

In [ ]:
coef_matrix

#Y=(exp_data/cell_size_factor-np.matmul(coef_matrix,x_data_covariate.T)[marker_unique_exp_data_idx]).transpose().astype(float)
(exp_data/cell_size_factor-np.exp(np.matmul(coef_matrix,x_data_null.T)))[marker_unique_exp_data_idx].transpose().astype(float)

In [ ]:
coef_matrix[0],x_data_null.T[:,0]

In [ ]:
np.matmul(coef_matrix,x_data_covariate.T)[marker_unique_exp_data_idx].shape

In [ ]:
np.exp(np.matmul(coef_matrix,x_data_null.T))

In [ ]:
#exp_data[marker_unique_exp_data_idx,:],Y#.shape

In [ ]:
Y.shape,np.matmul(coef_matrix,x_data_covariate.T)[marker_unique_exp_data_idx].shape

In [ ]:
np.matmul(coef_matrix,x_data_covariate.T)[marker_unique_exp_data_idx]

In [ ]:
coef_matrix

In [ ]:
coef_matrix.shape,coef_matrix

In [ ]:
np.array(list(map(lambda x: x[0],coef_list)))

In [ ]:
coef_matrix[:,0]

In [ ]:
cell_size_factor

In [ ]:
#### naive EM algorithm

# safe_log_sum_vector
@njit
def safe_log_sum_vector(vec):
    # safe_log_sum
    def safe_log_sum(logx, logy):
        if logx >= logy:
            return logx + np.log(1+np.exp(logy-logx))
        else:
            return logy + np.log(1+np.exp(logx-logy))
    
    log_sum = vec[-1]
    for i in prange(vec.shape[0]-1):
        log_sum = safe_log_sum(log_sum, vec[i])
    
    return log_sum

# expectaion for gamma
@njit(parallel=True)
def e_gamma(N, M, pi):
    log_gamma = np.zeros(shape=(N.shape[0], M.shape[1]))
    for i in prange(N.shape[0]): # prange if needed
        for t in range(pi.shape[0]):
            for f in range(N.shape[1]): # likelihood using sc special funcs
                log_gamma[i,t] += sc.xlogy(N[i,f],M[f,t]) - sc.gammaln(N[i,f]+1) - M[f,t]
            #log_gamma[i,t] += np.log(pi[t]) 
        log_gamma[i,:] = log_gamma[i,:] - safe_log_sum_vector(log_gamma[i,:])
    
    return np.exp(log_gamma)

# compute M
@njit(parallel=True)
def max_M(N, gamma):
    M = np.zeros(shape=(N.shape[1], gamma.shape[1]))
    for f in prange(N.shape[1]):
        for t in range(gamma.shape[1]):
             M[f,t] = np.dot(gamma[:,t],N[:,f])/np.sum(gamma[:,t])
    
    return M

# Poisson approximation
@jit(nopython=False)
def para_cell_poisson(N, n_type, n_iter=1000,M_init=None):
    """
    Initialization of parameters
    N, n_f^i: row are individuals, column are features
    """
    # gamma, gamma_t^i: row are individuals, column are types
    gamma = np.ones(shape=(N.shape[0], n_type)) / n_type
    # M, M_f^t: row are features, column are types 
    if M_init is None:
        M = (0.5+np.random.rand(N.shape[1], n_type)) * np.sum(N) / N.shape[0] / 10000
    else:
        M=M_init
    #M = np.ones(shape=(N.shape[1], n_type)) * np.sum(N) / N.shape[0]
    # pi, pi_t: vector of length n_type
    pi = np.ones(shape=n_type) / n_type
    
    # iteration
    for i in range(n_iter):
        # expectation step: update gamma
        gamma = e_gamma(N, M, pi)
        # maximization step
        pi = np.sum(gamma, axis=0) / N.shape[0]
        M = max_M(N, gamma) 
        
    return M, pi, gamma
    
# select Poisson or Negative-Binomial
def para_cell(N, n_type, likelihood='Poisson'):
    
    if likelihood == 'Poisson':
        M, pi, gamma = para_cell_poisson(N, n_type)
    
    return M, pi, gamma

In [ ]:
ll_series=pd.Series(ll_list,index=exp_data_row)
ll_series.apply(lambda x: np.log10(-x)).hist()

In [ ]:
#ll_series.apply(lambda x: np.log10(-x)).loc[np.unique([j for i in list(clustername_to_markers_new.values()) for j in i])].sort_values(ascending=False)

In [ ]:
#np.unique([j for i in list(clustername_to_markers_new.values()) for j in i])

In [ ]:
model_result.params

In [ ]:
model_result.summary()

In [ ]:
#dir(model_result)

In [ ]:
model_result.params

In [ ]:
model_result.summary()

In [ ]:
model_result.summary()

In [ ]:
print(exp_data_row[0],'is not in cell-type marker list')
y_data=exp_data[0,:]
model=sm.GLM(y_data,x_data_null,family=family)
model_result=model.fit()
model_result.summary()

In [ ]:
exp_data

In [ ]:
cell_size_factor

In [ ]:
family=sm.families.Poisson(link=sm.families.links.log())

### Convert data (exp_data,exp_data_col,exp_data_row) to input for for GLM

In [ ]:
x_data_covariate=x_data_covariate[:,:-1]#exclude the last col to avoid colinearity
x_data_covariate.shape

In [ ]:
x_data_null=np.concatenate([x_data_intercept,x_data_covariate],axis=1)

## Run regression

### test

In [ ]:
print(exp_data_row[0],'is not in cell-type marker list')
y_data=exp_data[0,:]
model=sm.GLM(y_data,x_data_null,family=family)
model_result=model.fit()
model_result.summary()

In [ ]:
pi_true=np.array([np.sum(exp_data_meta_clusterid_clusteridunique==i) for i in sorted(np.unique(exp_data_meta_clusterid_clusteridunique))])/exp_data_meta_clusterid_clusteridunique.shape[0]
M_true=np.array([np.mean(exp_data[marker_unique_idx,:][:,exp_data_meta_clusterid_clusteridunique==i],axis=1) for i in sorted(np.unique(exp_data_meta_clusterid_clusteridunique))])

In [ ]:
#pd.Series(exp_data_row).reset_index().set_index(0).loc[marker_unique]
#np.eye(len(marker_unique_idx))
#np.eye(len(marker_unique_idx))
clustername2marker_new_index=[[marker_unique.tolist().index(j) for j in i] for i in list(clustername2markers_new.values())]
M_init=np.array([np.sum(np.eye(len(marker_unique_idx))[idx],axis=0) for idx in clustername2marker_new_index])

# Performance Comparison

## cellassign

In [17]:
cellassign_prob=pd.read_csv('data/analysis/cellassign_fit_full2.prob.tsv',sep='\t')
#cellassign_prob=pd.read_csv('data/analysis/cellassign_fit.prob.tsv',sep='\t')
#cellassign_prob=pd.read_csv('data/analysis/cellassign_fit_full.prob.tsv',sep='\t')

In [25]:
sum(np.argmax(cellassign_prob.values,axis=1)==exp_data_meta_clusterid_clusteridunique)

7705

In [30]:
cellassign_prob_type=cellassign_prob.idxmax(axis=1)
#cellassign_prob_type[cellassign_prob.max(axis=1)<0.95]='other'

In [31]:
np.sum(cellassign_prob_type.values==clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values)

7705

In [32]:
np.unique(cellassign_prob_type.values,return_counts=True)

(array(['Cholangiocytes', 'Erythroid_cells', 'Hepatic_Stellate_Cells',
        'Hepatocytes', 'LSECs', 'Macrophages', 'Mature_B_cells',
        'NK_cells', 'Plasma_cells', 'ab_T_cells', 'gd_T_cells'],
       dtype=object),
 array([ 125,  121,   50, 3566,  725, 1172,  121,  608,  530,  772,  654]))

In [ ]:
cellassign_prob_type.unique()

In [ ]:
cellassign_prob_type[cellassign_prob.max(axis=1)<0.95]='other'

In [ ]:
cellassign_prob_type

In [ ]:

import numpy as np
a=test_prob_type;b=clusterid2clustername.loc[exp_data_meta['res.0.8'].values][1]

a[(a!='other').values].shape,np.sum(a[(a!='other').values]==b[(a!='other').values].values)#np.sum(a[a!='other'].values==b[a!='other'].values),np.sum(a[b!='other'].values==b[b!='other'].values)

In [ ]:
sorted(clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].unique())

In [ ]:
np.sum(cellassign_prob_type.values==clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values)

In [ ]:
np.sum(cellassign_prob_type.values==clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values)

In [ ]:
np.sum(cellassign_prob_type.values==clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values)

In [ ]:
np.sum(cellassign_prob_type.values==clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values)

In [ ]:
np.sum(cellassign_prob_type.values==clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values)

In [ ]:
np.sum(cellassign_prob_type.values==clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values)

In [ ]:
clusterid_to_clustername.loc[exp_data_meta['res.0.8'].values][1].values

In [ ]:
pi_true=np.array([np.sum(exp_data_meta_clusterid_clusteridunique==i) for i in sorted(np.unique(exp_data_meta_clusterid_clusteridunique))])/exp_data_meta_clusterid_clusteridunique.shape[0]
M_true=np.array([np.mean(exp_data[marker_unique_idx,:][:,exp_data_meta_clusterid_clusteridunique==i],axis=1) for i in sorted(np.unique(exp_data_meta_clusterid_clusteridunique))])

In [ ]:
#pd.Series(exp_data_row).reset_index().set_index(0).loc[marker_unique]
#np.eye(len(marker_unique_idx))
#np.eye(len(marker_unique_idx))
clustername2marker_new_index=[[marker_unique.tolist().index(j) for j in i] for i in list(clustername2markers_new.values())]
M_init=np.array([np.sum(np.eye(len(marker_unique_idx))[idx],axis=0) for idx in clustername2marker_new_index])

In [ ]:
#### naive EM algorithm

# safe_log_sum_vector
@njit
def safe_log_sum_vector(vec):
    # safe_log_sum
    def safe_log_sum(logx, logy):
        if logx >= logy:
            return logx + np.log(1+np.exp(logy-logx))
        else:
            return logy + np.log(1+np.exp(logx-logy))
    
    log_sum = vec[-1]
    for i in prange(vec.shape[0]-1):
        log_sum = safe_log_sum(log_sum, vec[i])
    
    return log_sum

# expectaion for gamma
@njit(parallel=True)
def e_gamma(N, M, pi):
    log_gamma = np.zeros(shape=(N.shape[0], M.shape[1]))
    for i in prange(N.shape[0]): # prange if needed
        for t in range(pi.shape[0]):
            for f in range(N.shape[1]): # likelihood using sc special funcs
                log_gamma[i,t] += sc.xlogy(N[i,f],M[f,t]) - sc.gammaln(N[i,f]+1) - M[f,t]
            #log_gamma[i,t] += np.log(pi[t]) 
        log_gamma[i,:] = log_gamma[i,:] - safe_log_sum_vector(log_gamma[i,:])
    
    return np.exp(log_gamma)

# compute M
@njit(parallel=True)
def max_M(N, gamma):
    M = np.zeros(shape=(N.shape[1], gamma.shape[1]))
    for f in prange(N.shape[1]):
        for t in range(gamma.shape[1]):
             M[f,t] = np.dot(gamma[:,t],N[:,f])/np.sum(gamma[:,t])
    
    return M

# Poisson approximation
@jit(nopython=False)
def para_cell_poisson(N, n_type, n_iter=1000,M_init=None):
    """
    Initialization of parameters
    N, n_f^i: row are individuals, column are features
    """
    # gamma, gamma_t^i: row are individuals, column are types
    gamma = np.ones(shape=(N.shape[0], n_type)) / n_type
    # M, M_f^t: row are features, column are types 
    if M_init is None:
        M = (0.5+np.random.rand(N.shape[1], n_type)) * np.sum(N) / N.shape[0] / 10000
    else:
        M=M_init
    #M = np.ones(shape=(N.shape[1], n_type)) * np.sum(N) / N.shape[0]
    # pi, pi_t: vector of length n_type
    pi = np.ones(shape=n_type) / n_type
    
    # iteration
    for i in range(n_iter):
        # expectation step: update gamma
        gamma = e_gamma(N, M, pi)
        # maximization step
        pi = np.sum(gamma, axis=0) / N.shape[0]
        M = max_M(N, gamma) 
        
    return M, pi, gamma
    
# select Poisson or Negative-Binomial
def para_cell(N, n_type, likelihood='Poisson'):
    
    if likelihood == 'Poisson':
        M, pi, gamma = para_cell_poisson(N, n_type)
    
    return M, pi, gamma

In [ ]:
M_output,pi_output,gamma_output =para_cell_poisson(exp_data[marker_unique_idx,:].transpose().astype(float),11)

In [ ]:
gamma_output_argmax=np.argmax(gamma_output,axis=1)

In [ ]:
output_true_matching=[np.argmax([pearsonr(value,value2)[0] for value2 in M_true]) for value in M_output.transpose()]
output_true_matching

In [ ]:
#output_true_matching[gamma_output_argmax]

In [ ]:
gamma_output_argmax_match=[output_true_matching[idx] for idx in gamma_output_argmax]

In [ ]:
np.sum(exp_data_meta_clusterid_clusteridunique==gamma_output_argmax_match)#cluster#gamma_output_argmax_match

In [ ]:
shape(500*M_init),500*M_init

In [ ]:
#np.sum(M_output,axis=0),500*np.sum(M_init,axis=0)

In [ ]:
M_output,pi_output,gamma_output =para_cell_poisson(exp_data[marker_unique_idx,:].transpose().astype(float),11,M_init=500*M_init)

In [ ]:
gamma_output_argmax=np.argmax(gamma_output,axis=1)

In [ ]:
pi_output=exp_data

In [ ]:
gamma_output_argmax,pi_output,gamma_output,M_output

In [ ]:
gamma_output_argmax_match=[output_true_matching[idx] for idx in gamma_output_argmax]

In [ ]:
np.sum(exp_data_meta_clusterid_clusteridunique==gamma_output_argmax_match)#cluster#gamma_output_argmax_match

In [ ]:
pi_true*8444

In [ ]:
clustername2marker_new_index

In [ ]:
len(gamma_output_argmax_match)

In [ ]:
exp_data

In [ ]:
plt.hist(np.mean(exp_data,axis=0),bins=30)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
M_output_,exp_output_,gamma_output_ =para_cell_poisson(exp_data[marker_unique_idx,:].transpose().astype(float),11)

In [ ]:
exp_output_

In [ ]:
exp_M, exp_pi, exp_gamma =para_cell_poisson(exp_data[marker_unique_idx,:].transpose().astype(float),12,np.concatenate((cluster_M.transpose(),np.random.rand(63,1)),axis=1))

In [ ]:
cluster_M.transpose().shape

In [ ]:
np.concatenate((cluster_M.transpose(),np.random.rand(63,1)),axis=1)

In [ ]:
exp_M

In [ ]:
exp_data[marker_unique_idx,:].transpose()

In [ ]:
np.sum(cluster_M),np.sum(cluster_M,axis=0)#,np.sum(cluster_M,axis=1),

In [ ]:
np.sum(cluster_M),np.sum(exp_M),cluster_M.shape,exp_M.shape

In [ ]:
np.sum(exp_M),np.sum(exp_M,axis=0)

In [ ]:
exp_data[marker_unique_idx,:].transpose().astype(float).shape

In [ ]:
exp_M, exp_pi, exp_gamma =para_cell_poisson(exp_data[marker_unique_idx,:].transpose().astype(float),11)

In [ ]:
#exp_M

In [ ]:
exp_cluster_true_matching=[np.argmax([pearsonr(value,value2)[0] for value2 in cluster_true]) for value in exp_M.transpose()]
exp_cluster_true_matching
np.unique(exp_cluster_true_matching)

In [ ]:
np.array([exp_cluster_true_matching[i] for i in np.argmax(exp_gamma,axis=1)]).shape

In [ ]:
#exp_data_meta_clusterid_unique.values.shape
is_match=exp_data_meta_clusterid_unique.values==np.array([exp_cluster_true_matching[i] for i in np.argmax(exp_gamma,axis=1)])

In [ ]:
for i in enumerate(is_match):
    print(i[1])
    if i[1]>100:
        break

In [ ]:
#exp_cluster[np.argmax(exp_gamma,axis=1).tolist()]
#np.argmax(exp_gamma,axis=1).tolist()
#exp_data_meta_clusterid_unique.values.shape
print(np.sum(is_match))

In [ ]:
np.unique(exp_cluster)

In [ ]:
np.argmax(exp_gamma,axis=1)

In [ ]:
exp_cluster#.cluster_count

In [ ]:
clusterid2clustername.loc[exp_data_meta['res.0.8'].values][1].unique()

In [ ]:
cluster_M=np.array([np.mean(exp_data[marker_unique_idx,:][:,exp_data_meta_clusterid_unique==i],axis=1) for i in sorted(np.unique(exp_data_meta_clusterid_unique))])
cluster_count=np.array([np.sum(exp_data_meta_clusterid_unique==i) for i in sorted(np.unique(exp_data_meta_clusterid_unique))])

In [ ]:
len(clusterid2clustername[1].unique().tolist()),len(np.unique(exp_data_meta_clusterid_unique))

In [ ]:
len(exp_data_meta['res.0.8'].unique())

In [ ]:
len([pearsonr(value,value2)[0] for value2 in cluster_true])

In [ ]:
[np.max([pearsonr(value,value2)[0] for value2 in cluster_true]) for value in exp_M.transpose()]

In [ ]:
cluster_true.shape#[0].shape